In [ ]:
# import module
from sklearn.model_selection import train_test_split
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
import logging.handlers
import datetime
import argparse
import logging.handlers

class TuneParams():

    def __init__(self):
        self.parser = argparse.ArgumentParser(description='Tune Parameters...')

        # data
        self.parser.add_argument('--batch_size', default=32, type=int, help='number of batch size')

        # test rate
        self.parser.add_argument('--rate', default=0.1, type=float, help='split rate(default : 0.1)')

        # epoch
        self.parser.add_argument('--epochs', type=int, default=5, help='number of total training epochs')

        # learning rate
        self.parser.add_argument('--lr', '--learning-rate', default=0.1, type=float,
                                 help='learning rate(deafult : 0.1)')

        # optimizer & scheduler(various choices)
        self.parser.add_argument('--optim', type=str, default='sgd', choices=['sgd', 'adam'])
        self.parser.add_argument('--lr_scheduler', default='none', type=str, choices=['multistep','stepLR','LambdaLR', 'none'])

        self.args = self.parser.parse_args(args=[])

    def create_optim(self, params):
        if self.args.optim == 'sgd':
            optimizer = optim.SGD(params, lr=self.args.lr)
        elif self.args.optim == 'adam':
            optimizer = optim.Adam(params, lr=self.args.lr, weight_decay=0.0)

        return optimizer

    def create_scheduler(self, optimizer):
        if self.args.lr_scheduler == 'multistep':
            scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20,35,45], gamma=0.1)
        elif self.args.lr_scheduler == 'stepLR':
            scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
        elif self.args.lr_scheduler == 'LambdaLR':
            scheduler= optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                                    lr_lambda=lambda epoch: 0.95 ** epoch,
                                                    last_epoch=-1,
                                                    verbose=False)
        elif self.args.lr_scheduler == 'none':
            scheduler = None

        else:
            raise ValueError("No Scheduler\n")

        return scheduler
    
    def custom_input(self,lr_list=[0.1],optim_list=['sgd'],scheduler_list=['multistep'],test_ratio_list=[0.1]):
        # result stores accuracy, args, index(iterate each learning rate, optimizer, scheduler)
        # ex) 59.8333, "Namespace(batch_size=32, epochs=10, lr=0.1, lr_scheduler='multistep', optim='sgd')" ...
        result = []
        i = 1  # index stores the best model's index

        # ========== set logger ==========
        logger = logging.getLogger('logger')
        fomatter = logging.Formatter('[%(levelname)s| %(filename)s:%(lineno)s] >> %(message)s')
        time = datetime.datetime.now().strftime("%m%d_%H:%M")
        fileHandler = logging.FileHandler('./log_' + time + '.txt')
        fileHandler.setFormatter(fomatter)
        logger.addHandler(fileHandler)
        logger.setLevel(logging.DEBUG)
        # ================================

        # execute possibe combinations
        # to get highest performance combination and that is the best params
        for rate in range(len(test_ratio_list)):
            for a in range(len(lr_list)):
                for b in range(len(optim_list)):
                    for c in range(len(scheduler_list)):
                        self.args.rate = test_ratio_list[rate]
                        self.args.lr = lr_list[a]
                        self.args.optim = optim_list[b]
                        self.args.lr_scheduler = scheduler_list[c]

                        print(self.args)

                        # info about possible combinations
                        logger.info("========================================")
                        logger.info(
                            "batch_size: {}, total epoch: {}, test data ratio: {}, initial learning rate: {}, optimizer: {}, scheduler: {} "
                                .format(self.args.batch_size, self.args.epochs, self.args.rate, self.args.lr,
                                        self.args.optim, self.args.lr_scheduler))

                        argString = str(self.args)
                        accuracy = testing(self, i, logger)
                        result.append((accuracy, argString, i))
                        i = i + 1

        print('Before sorting')
        print(result)
        result.sort(key=lambda x: x[0], reverse=True)

        print('After sorting')
        print(result)
        print(result[0][1])

        model_pth = "./best_model_[" + str(result[0][2]) + "].pth"  # ex) best_model_[8].pth
        print(model_pth)
